In [2]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import transforms
from PIL import Image
import numpy as np
import seaborn as sns
import torch
from sklearn.metrics import precision_recall_fscore_support
import torchvision.transforms as transforms
import torchvision.models as models
import numpy  as np
#from cifar_dataloader import dataloader1
from train_evaluate import train,validate,evaluate
import numpy as np
import seaborn as sns
import torch
import torchvision.models as models
import torch.nn as nn
import importlib
import  cifar_dataloader
importlib.reload(cifar_dataloader)
#import  cifar_dataloader
from cifar_dataloader import dataloader2



sns.set()

sns.set()

In [3]:
class DatasetArray(Dataset):
    def __init__(self,features,labels,transform):
        self.features=features
        self.labels=labels
        self.transform=transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,index):
        features=self.features[index]
        labels=self.labels[index]
        #features = Image.fromarray(features.astype(np.uint8))
        features_np = features.numpy()

        # Convert NumPy array to PIL Image
        features= Image.fromarray((features_np * 255).astype('uint8').transpose(1, 2, 0))
        
        if self.transform is not None:
            features=self.transform(features)
            #print(features.shape)
        return (features,labels)
    

def dataloader2(dataset):
    train_features=dataset['Xtr']
    #train_features=train_features
    train_labels=dataset['Str']
    train_labels = torch.from_numpy(train_labels)
    train_labels=torch.nn.functional.one_hot(train_labels, num_classes=3).float()
    test_features=dataset['Xts']
    test_labels=dataset['Yts']
    test_labels = torch.from_numpy(test_labels)
    test_labels=torch.nn.functional.one_hot(test_labels, num_classes=3).float()
    x_train,x_val,y_train,y_val=train_test_split(train_features[:500],train_labels[:500],test_size=0.2)
    print(x_train.shape)
    x_train=torch.from_numpy(x_train.astype(np.float32) / 255).permute(0, 3, 1, 2)
    x_val=torch.from_numpy(x_val.astype(np.float32) / 255).permute(0, 3, 1, 2)
    x_test=torch.from_numpy(test_features.astype(np.float32) / 255).permute(0, 3, 1, 2)
    #batch load
    data_transform = {
    "train": transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    #transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]),
    "val": transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]),

    "test":transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()])}
    train_dataset=DatasetArray(x_train,y_train,transform=data_transform["train"])
    val_dataset=DatasetArray(x_val,y_val,transform=data_transform["val"])
    # train_dataset=DatasetArray(x_train,y_train)
    # val_dataset=DatasetArray(x_val,y_val)
    test_data=DatasetArray(x_test,test_labels,transform=data_transform["test"])
    
    train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
    val_loader=DataLoader(val_dataset,batch_size=64,shuffle=True)
    test_loader=DataLoader(test_data,batch_size=64,shuffle=True)

    return train_loader,val_loader,test_loader



In [4]:


def train(model, criterion, optimizer, training_dataloader):
    sum_accuracy = 0
    sum_loss = 0
    count = 0
    model.train()
    for features, labels in training_dataloader:
        probabilities=model(features)
        loss=criterion(probabilities,labels)
        #print(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        mini_batch_size=labels.size(0)
        labels=torch.argmax(labels, dim=1)
        accuracy=(
            torch.sum(torch.max(probabilities,dim=1).indices==labels)
        )

        sum_accuracy+=accuracy.item()
        sum_loss+=loss.item()*mini_batch_size
        #print(sum_loss)
        count+=mini_batch_size
        #print(sum_loss/count)
        train_acc=sum_accuracy/count
        train_loss=sum_loss/count
    return train_acc, train_loss


def validate(model,criterion,dataloader):
    sum_accuracy=0
    sum_loss=0
    count=0
    model.eval()
    for features ,labels in dataloader:
        
        with torch.no_grad():
            probabilities=model(features)
            loss=criterion(probabilities,labels)
        mini_batch_size=labels.size(0)
        labels=torch.argmax(labels, dim=1)
        accuracy=(
            torch.sum(torch.max(probabilities,dim=1).indices==labels)
        )

        sum_accuracy+=accuracy.item()
        sum_loss+=loss.item()*mini_batch_size
        #print(sum_loss)
        count+=mini_batch_size
        #print(sum_loss/count)
        val_acc=sum_accuracy/count
        val_loss=sum_loss/count
    return val_acc, val_loss
    
def evaluate(dataset,model):
    # x_test=dataset["Xts"]
    # y_test=dataset["Yts"]
    # x_test=torch.from_numpy(x_test.astype(np.float32) / 255).permute(0, 3, 1, 2)
    # cifar_xtest=torch.Tensor(x_test)
    # print(cifar_xtest.shape)
    # y_pred=torch.max(model(cifar_xtest),dim=1).indices
    # correct=0
    # for index , value in enumerate(y_pred):
    #     if value == y_test[index]:
    #         correct += 1
    # #report=classification_report(y_test,y_pred.numpy())
    # precision,recall,_,_=precision_recall_fscore_support(y_test,y_pred,average='weighted')

    correct=0
    sum_count=0
    precision_all=0
    recall_all=0
    times=0
    model.eval()
    for features , labels in dataset:
        with torch.no_grad():
            probabilities=model(features)
            y_pred=torch.max(probabilities,dim=1).indices
            for index , value in enumerate(y_pred):
                if value == labels[index]:
                    correct += 1
            sum_count+=len(y_pred)
            precision,recall,_,_=precision_recall_fscore_support(labels,y_pred,average='weighted')
            precision_all+=precision
            recall_all+=recall
            times+=1
    
    accuracy=correct/sum_count
    precision=precision_all/times
    recall=recall_all/times

    return precision, recall,accuracy


        


In [5]:
CIFAR = np.load('./datasets/CIFAR.npz')
train_loader,val_loader,test_loader=dataloader2(CIFAR)

(400, 32, 32, 3)


In [6]:
weights = models.ViT_B_16_Weights.DEFAULT
vit = models.vit_b_16(weights=weights)

In [7]:
class VIT(nn.Module):
    def __init__(self, output_shape):
        super(VIT, self).__init__()

        self.pretrained = models.vit_b_16(weights=weights)
        for param in self.pretrained.parameters():
            param.requires_grad = False

        pretrained_output = 1000
        self.fc = torch.nn.Sequential(
            nn.Linear(in_features=pretrained_output,
                      out_features=output_shape,
                      bias=False),
        )

    def forward(self, image):
        output = self.pretrained(image)
        output = self.fc(output)
        return output

vit = VIT(output_shape=3)

In [8]:
def run(dataset,epochs=10,lr=0.01):

    train_loader,val_loader,test_loader=dataloader2(dataset)
    #model=resnet18
    #model=Lenet()
    model=vit
    # resnet18 = resnet18.fc.in_features
    # resnet18.fc = nn.Linear(num_ftrs, 3)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
    training_losses = []
    training_accuracies = []
    validation_losses = []
    validation_accuracies = []
    test_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        train_acc, train_loss=train(model,criterion, optimizer, train_loader)
        val_acc, val_loss=validate(model,criterion, val_loader)
        test_acc, test_loss=validate(model,criterion, test_loader)

        training_accuracies.append(train_acc)
        training_losses.append(train_loss)
        validation_accuracies.append(val_acc)
        validation_losses.append(val_loss)
        test_accuracies.append(test_acc)
        test_losses.append(test_loss)


        print(f"epoch: {epoch}\n \
            train_acc: {train_acc}, train_loss: {train_loss}\n \
            val_acc: {val_acc}, val_loss: {val_loss}\n \
            test_acc: {test_acc}, test_loss: {test_loss}\n")
    precision,recall=evaluate(dataset,model)


    return (training_accuracies,training_losses,
            validation_accuracies,
            validation_losses,
            test_accuracies,
            test_losses,
            precision,
            recall,
            model)


        



In [ ]:
(training_accuracies,training_losses,
validation_accuracies,
validation_losses,
test_accuracies,
test_losses,
precision,
recall,
model)=run(CIFAR,epochs=1,lr=0.01)